In [122]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

This notebook focuses on selecting techniques for the final pipeline. For this initial model approach, various techniques are evaluated through cross-validation using a basic Logistic Regression model with default settings:

- If one technique demonstrates significantly better performance, it will be incorporated into the pipeline.
- If no significant difference is observed, the simplest technique will be selected.

Data indexes are tracked because some data is stored as a sparse NumPy array. Although the split was performed using a seed, consistency across library versions or operating systems cannot be guaranteed.

The **f1_macro** metric is used for evaluation, as it provides a balanced assessment of false positives and false negatives, ensuring equal consideration for both categories.

In [21]:
df = pd.read_csv('processed_text_with_all.csv')
df = df.dropna()

train_index = pd.read_csv('train_index.csv').values[:, 0]
test_index = pd.read_csv('test_index.csv').values[:, 0]

In [22]:
X_train = df.Text.loc[train_index]
X_test = df.Text.loc[test_index]
y_train = df.Labels.loc[train_index]
y_test = df.Labels.loc[test_index]

# Text to vector

This notebook explores different algorithms to convert processed text into vectors: CountVectorizer, TF-IDF, and Word2Vec. The Word2Vec training process is handled in the Word2VecTraining notebook, and here, the pre-trained data will be loaded.

- **300 features** have been chosen for all models to balance memory usage and avoid overfitting, maintaining a reasonable ratio between rows and columns. The models will be compared based on these 300 features.
- For both CountVectorizer and TF-IDF, 1-3 n-grams will be trained. The 2-gram model will capture word pairs like "not" and other composite terms, while the 3-gram model will ensure no useful information is overlooked.

In [23]:
count_vect3 = CountVectorizer(ngram_range=(1, 3), max_features=300)
X_train_cv3 = count_vect3.fit_transform(X_train) 
# X_val_cv = count_vect.transform(X_val)
X_test_cv3 = count_vect3.transform(X_test)

In [70]:
count_vect2 = CountVectorizer(ngram_range=(1, 2), max_features=300)
X_train_cv2 = count_vect2.fit_transform(X_train) 
# X_val_cv = count_vect.transform(X_val)
X_test_cv2 = count_vect2.transform(X_test)

In [86]:
count_vect1 = CountVectorizer(ngram_range=(1, 1), max_features=300)
X_train_cv1 = count_vect1.fit_transform(X_train) 
# X_val_cv = count_vect.transform(X_val)
X_test_cv1 = count_vect1.transform(X_test)

In [89]:
count_vect2.vocabulary_ == count_vect3.vocabulary_

True

In [110]:
count_vect2.vocabulary_ == count_vect1.vocabulary_

False

In [24]:
tfidf3 = TfidfVectorizer(ngram_range=(1, 3), max_features=300)
X_train_tfidf3 = tfidf3.fit_transform(X_train)
# X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf3 = tfidf3.transform(X_test)

In [71]:
tfidf2 = TfidfVectorizer(ngram_range=(1, 2), max_features=300)
X_train_tfidf2 = tfidf2.fit_transform(X_train)
# X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf2 = tfidf2.transform(X_test)

In [90]:
tfidf1 = TfidfVectorizer(ngram_range=(1, 1), max_features=300)
X_train_tfidf1 = tfidf1.fit_transform(X_train)
# X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf1 = tfidf1.transform(X_test)

In [91]:
tfidf3.vocabulary_ == tfidf2.vocabulary_

True

In [92]:
tfidf2.vocabulary_ == tfidf1.vocabulary_

False

In [50]:
X_train_w2v = pd.read_csv('w2v_window10_train_data.csv', index_col=0)
X_test_w2v = pd.read_csv('w2v_window10_test_data.csv', index_col=0)

In [93]:
train_data_from_models = {'count_vec_ngram1': {'x': X_train_cv1,
                                               'y': y_train},
                          'tfidf_ngram1': {'x': X_train_tfidf1,
                                           'y': y_train},
                          'count_vec_ngram2': {'x': X_train_cv2,
                                               'y': y_train},
                          'tfidf_ngram2': {'x': X_train_tfidf2,
                                           'y': y_train},
                          'word2vec': {'x': X_train_w2v,
                                       'y': y_train}}

In [ ]:
from sklearn.model_selection import cross_val_score


def generate_cross_scoring_table_from_transf_data(train_data_dict, 
                                                  pred_model, scoring='f1_macro', cv=10):

    all_scores = [cross_val_score(pred_model, train_data['x'], 
                                  train_data['y'], cv=cv, scoring=scoring)
                  for train_data in train_data_dict.values()]

    return pd.DataFrame(all_scores, index=train_data_dict.keys())

In [95]:
lr = LogisticRegression()


result = generate_cross_scoring_table_from_transf_data(train_data_from_models, 
                                                       lr, scoring='f1_macro', cv=10)


result

,0,1,2,3,4,5,6,7,8,9
count_vec_ngram1,0.730720,0.744704,0.740865,0.744642,0.733056,0.746572,0.741397,0.748263,0.743674,0.738538
tfidf_ngram1,0.747909,0.759543,0.754347,0.762461,0.754868,0.759464,0.753718,0.760397,0.758010,0.751561
count_vec_ngram2,0.734709,0.746985,0.743778,0.748821,0.737387,0.747985,0.744234,0.749215,0.743730,0.741885
tfidf_ngram2,0.751512,0.761197,0.756324,0.765519,0.757312,0.760695,0.753588,0.762479,0.759185,0.753557
word2vec,0.819960,0.833779,0.826369,0.834525,0.830219,0.830770,0.827782,0.834607,0.831262,0.830440


The word2vec is clearly better than the rest, so we will keep the data and the model from the word2vec for the following steps.

In [24]:
import pandas as pd

a = """0	1	2	3	4	5	6	7	8	9
count_vec_ngram1	0.730720	0.744704	0.740865	0.744642	0.733056	0.746572	0.741397	0.748263	0.743674	0.738538
tfidf_ngram1	0.747909	0.759543	0.754347	0.762461	0.754868	0.759464	0.753718	0.760397	0.758010	0.751561
count_vec_ngram2	0.734709	0.746985	0.743778	0.748821	0.737387	0.747985	0.744234	0.749215	0.743730	0.741885
tfidf_ngram2	0.751512	0.761197	0.756324	0.765519	0.757312	0.760695	0.753588	0.762479	0.759185	0.753557
word2vec	0.819960	0.833779	0.826369	0.834525	0.830219	0.830770	0.827782	0.834607	0.831262	0.830440""".split("\n")


b = [a1.split("\t") for a1 in a]

columns = b[0]
columns
c = pd.DataFrame(b[1:])
c.index = c[0]
c.drop(0, axis=1, inplace=True)
c.columns = columns
c = c.astype(float)
c.mean(axis=1).idxmax()

'word2vec'

# Handle imbalance labels

Simple random undersampling, random oversampling, and SMOTE will be tested to evaluate if these techniques improve the model's performance.

A different cross-validation technique will be used to ensure the model is evaluated on data that has not been sampled, avoiding any potential bias.

In [62]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [112]:
X_us_res, y_us_res = RandomUnderSampler().fit_resample(X_train_w2v, y_train)

In [114]:
X_os_res, y_os_res = RandomOverSampler().fit_resample(X_train_w2v, y_train)

In [115]:
X_sm_res, y_sm_res = SMOTE().fit_resample(X_train_w2v, y_train)

In [125]:
imb_models = {'base': None,
              'undersampling': RandomUnderSampler(),
              'oversampling': RandomOverSampler(),
              'smote': SMOTE()}

In [ ]:
from sklearn.model_selection import ShuffleSplit


def generate_imb_results_table(pred_model_class, imb_models, X_train_orig, y_train_orig):
    all_results = []
    for model in imb_models.values():
        ss = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
        ind_results = []
        for train_index, test_index in ss.split(X_train_orig.values):
            x_train = X_train_orig.iloc[train_index]
            y_train = y_train_orig.iloc[train_index]
            x_test = X_train_orig.iloc[test_index]
            y_test = y_train_orig.iloc[test_index]

            if model:
                x_train_imb, y_train_imb = model.fit_resample(x_train, y_train)
            else:
                x_train_imb, y_train_imb = x_train, y_train
            pred_model = pred_model_class()
            pred_model.fit(x_train_imb, y_train_imb)
            y_pred = pred_model.predict(x_test)
            ind_results.append(f1_score(y_test, y_pred, average='macro'))
            
        all_results.append(ind_results)
    return pd.DataFrame(all_results, index=imb_models.keys())

In [159]:
imb_result = generate_imb_results_table(LogisticRegression, imb_models, X_train_w2v, y_train)

In [160]:
imb_result

,0,1,2,3,4,5,6,7,8,9
base,0.830428,0.831779,0.830530,0.832414,0.826815,0.827950,0.828817,0.826058,0.829886,0.828536
undersampling,0.804243,0.807803,0.801824,0.802430,0.803316,0.801522,0.803760,0.802825,0.803799,0.803864
oversampling,0.804965,0.808338,0.803273,0.802389,0.803654,0.803031,0.805854,0.803048,0.804639,0.804271
smote,0.810347,0.814086,0.809512,0.807804,0.808141,0.808434,0.809850,0.808486,0.809418,0.808970


In [ ]:
from scipy.stats import ttest_ind, wilcoxon


def test_ditr_significantly_greater_than_another(data1, data2, test_type, alpha=0.05):

    print(f'----------------------- {test_type.capitalize()} -----------------------')

    if test_type == 'ttest':
        test_func = ttest_ind
    elif test_type == 'wilcoxon':
        test_func = wilcoxon
    else:
        raise ValueError(f'test_type must be ttest or wilcoxon not {test_type}')

    statistic, p_value = test_func(data1, data2, alternative='greater')
    
    if p_value < alpha:
        print("The mean of the first distribution is significantly" +
            " greater than the mean of the second one.")
    else:
        print("The mean of the first distribution is not significantly" +
            " greater than the mean of the second one.")
        
    return statistic, p_value

In [161]:
statistic, p_value = ttest_ind(imb_result.loc['base'],
                               imb_result.loc['oversampling'], 
                               alternative='greater')

test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['oversampling'], 
                                             'ttest')
    
test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['oversampling'], 
                                             'wilcoxon')

----------------------- Ttest -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.
----------------------- Wilcoxon -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.


(55.0, 0.0009765625)

In [162]:
test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['undersampling'], 
                                             'ttest')
    
test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['undersampling'], 
                                             'wilcoxon')

----------------------- Ttest -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.
----------------------- Wilcoxon -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.


(55.0, 0.0009765625)

In [163]:
test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['smote'], 
                                             'ttest')
    
test_ditr_significantly_greater_than_another(imb_result.loc['base'], 
                                             imb_result.loc['smote'], 
                                             'wilcoxon')

----------------------- Ttest -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.
----------------------- Wilcoxon -----------------------
The mean of the first distribution is significantly greater than the mean of the second one.


(55.0, 0.0009765625)

It seems like for this dataset, the imbalancing techniques worsen the results. Therefore, we are skipping them.